In [8]:
import numpy as np

## Definitions:

### Primitive root of unity
Any complex number that yields 1 when raised to some positive integer power n.
1. $ \zeta^n = 1 $
2. $ \zeta^k \neq 1 $ for all integers $ k $ such that $ 0 < k < n $.

$\zeta^n = e^{i(2\pi \frac{k}{n})}$

### Cyclotomic Polynomial
A **cyclotomic polynomial** is a special type of polynomial defined as the unique irreducible polynomial with integer coefficients whose roots are the **primitive roots of unity** $e^{i(2\pi\frac{k}{n})}$, where k runs over the positive integers less than n and coprime to n. The formula is equal to:

$$
\Phi_{n}(x) = \prod_{\substack{1 \leq k \leq n \\ \gcd(k, n) = 1}} \left(x - e^{2i\pi \frac{k}{n}}\right)
$$

The $n$-th cyclotomic polynomial, denoted as $\Phi_n(x)$, is defined by:



In [2]:
class CKKSEncoder:
    """Basic CKKS encoder to encode complex vectors into polynomials."""
    
    def __init__(self, M: int):
        """Initialization of the encoder for M a power of 2. 
        
        xi, which is an M-th root of unity will, be used as a basis for our computations.
        """
        self.xi = np.exp(2 * np.pi * 1j / M)
        self.M = M

In [7]:
# First we set the parameters
M = 8
N = M //2

# We set xi, which will be used in our computations
xi = np.exp(2 * np.pi * 1j / M)
print(xi)
encoder = CKKSEncoder(M)

b = np.array([1, 2, 3, 4])

(0.7071067811865476+0.7071067811865475j)
